--------------

**This notebook is adapted from that in [pysal/spaghetti](https://github.com/pysal/spaghetti/blob/master/tools/gitcount.ipynb), which was in turn adapted from [pysal/pysal](https://github.com/pysal/pysal/blob/master/tools/gitcount.ipynb).**

---------------

## Change Log Statistics

This notebook generates the summary statistics for a package. It assumes you are running this under the `tools` directory at the toplevel of the package. This notebook will generate a file in the current directory with the name ``changelog.md``. You can edit and append this on front of the ``CHANGELOG.md`` file for the package release.

## Change the values only in the next cell

In [ ]:
directory = "jGaboardi"
package_name = "nhgisxwalk"

---------------------------

In [ ]:
import os
import json
import re
import ssl
from collections import Counter
from datetime import datetime, timedelta
from subprocess import check_output, Popen, PIPE
from time import sleep
from urllib.request import urlopen
context = ssl._create_unverified_context()
CWD = os.path.abspath(os.path.curdir)

### Helper functions

In [ ]:
def regularize_identity(string):
    string = string.decode()
    if len(string.split(" "))>1:
        string = string.title()
    return string.lstrip("* ")

def parse_link_header(headers):
    link_s = headers.get("link", "")
    urls = element_pat.findall(link_s)
    rels = rel_pat.findall(link_s)
    d = {}
    for rel,url in zip(rels, urls):
        d[rel] = url
    return d

def get_paged_request(url):
    """Get a full list, handling APIv3's paging."""
    results = []
    while url:
        f = urlopen(url)
        results.extend(json.load(f))
        links = parse_link_header(f.headers)
        url = links.get("next")
    return results

def get_issues(project, state="closed", pulls=False):
    """Get a list of the issues from the Github API."""
    which = "pulls" if pulls else "issues"
    url = "https://api.github.com/repos/%s/%s?state=%s&per_page=%i" % (
        project, which, state, PER_PAGE
    )
    return get_paged_request(url)

def _parse_datetime(s):
    """Parse dates in the format returned by the Github API."""
    if s:
        return datetime.strptime(s, ISO8601)
    else:
        return datetime.fromtimestamp(0)

def issues2dict(issues):
    """Convert a list of issues to a dict, keyed by issue number."""
    idict = {}
    for i in issues:
        idict[i["number"]] = i
    return idict

def is_pull_request(issue):
    """Return True if the given issue is a pull request."""
    return "pull_request_url" in issue

def issues_closed_since(
        period=timedelta(days=365), project=None, pulls=False
    ):
    """Get all issues closed since a particular point in time. The
    period parameter can either be a datetime object, or a timedelta object.
    In the latter case, it is used as a time before the present. Rejected
    pull requests are excluded.
    """
    which = "pulls" if pulls else "issues"
    if isinstance(period, timedelta):
        period = datetime.now() - period
    url = "https://api.github.com/repos/"
    url += "%s/%s?state=closed&sort=updated&since=%s&per_page=%i" % (
        project, which, period.strftime(ISO8601), PER_PAGE
    )
    allclosed = get_paged_request(url)
    filtered = [
        i for i in allclosed if _parse_datetime(i["closed_at"]) > period
    ]
    if pulls:
        filtered = [ pr for pr in filtered if pr["merged_at"] ]
    return filtered

def sorted_by_field(issues, field="closed_at", reverse=False):
    """Return a list of issues sorted by closing date date."""
    return sorted(issues, key = lambda i:i[field], reverse=reverse)

def report(issues, show_urls=False):
    """Summary report about a list of issues, printing number and title.
    Titles may have unicode in them, so we must encode everything below.
    """
    if show_urls:
        for i in issues:
            role = "ghpull" if "merged_at" in i else "ghissue"
            print(
                "* :%s:`%d`: %s" % (
                    role, i["number"], i["title"].encode("utf-8")
                )
            )
    else:
        for i in issues:
            print("* %d: %s" % (i["number"], i["title"].encode("utf-8")))

###  Get the date of the last tag

In [ ]:
x, err = Popen(
    'git log -1 --tags --simplify-by-decoration --pretty="%ai"| cat',
    stdin=PIPE,
    stdout=PIPE,
    stderr=PIPE,
    shell=True
).communicate()
start_date = x.split()[0].decode("utf-8")

###  Get today's date

In [ ]:
release_date = str(datetime.today()).split()[0]

### Calculate time since last tagged release

In [ ]:
since_date = '--since="%s"' % start_date
since = datetime.strptime(start_date+" 0:0:0", "%Y-%m-%d %H:%M:%S")
since

### Get current version

In [ ]:
f = "../%s/__init__.py" % package_name
with open(f, "r") as initfile:
     exec(initfile.readline())  

### Total commits

In [ ]:
cmd = ["git", "log", "--oneline", since_date]
ncommits = len(check_output(cmd).splitlines())
ncommits

### List Contributors

In [ ]:
author_cmd = ["git", "log", "--format=* %aN", since_date]
ncommits = len(check_output(cmd).splitlines())
all_authors = check_output(author_cmd).splitlines()
counter = Counter([regularize_identity(author) for author in all_authors])
unique_authors = sorted(set(all_authors))
unique_authors = counter.keys()
unique_authors

### Disaggregate by PR, Issue

In [ ]:
ISO8601 = "%Y-%m-%dT%H:%M:%SZ"
PER_PAGE = 100
element_pat = re.compile(r"<(.+?)>")
rel_pat = re.compile(r'rel=[\'"](\w+)[\'"]')

### Fetch pulls and issues

In [ ]:
all_issues = {}
all_pulls = {}
total_commits = 0
prj = "%s/%s" % (directory, package_name)
issues = issues_closed_since(since, project=prj, pulls=False)
pulls = issues_closed_since(since, project=prj, pulls=True)
issues = sorted_by_field(issues, reverse=True)
pulls = sorted_by_field(pulls, reverse=True)
n_issues, n_pulls = map(len, (issues, pulls))
n_total = n_issues + n_pulls

### Generate issue listing

In [ ]:
issue_listing = []
for issue in issues:
    entry = "%s (#%s)" % (issue["title"], issue["number"])
    issue_listing.append(entry)
issue_listing

### Generate pull listing

In [ ]:
pull_listing = []
for pull in pulls:
    entry = "%s (#%s)" % (pull["title"], pull["number"])
    pull_listing.append(entry)
pull_listing

### Generate mesage

In [ ]:
message = "We closed a total of "
message += "%s issues (enhancements and bug fixes) " % n_total
message += "through %s pull requests, " % n_pulls
message += "since our last release on %s." % str(start_date)
message += "\n\n## Issues Closed\n"
issues = "\n".join(["  - "+issue for issue in issue_listing])
message += issues
message += "\n\n## Pull Requests\n"
pulls = "\n".join(["  - "+pull for pull in pull_listing])
message += pulls
people = "\n".join(["  - "+person for person in unique_authors])
message += "\n\nThe following individuals contributed to this release: "
message += "\n\n%s" % people
head = "# Changes\n\nVersion %s (%s)\n\n" % (__version__, release_date)
message = head + message
message

### Write out `changelog.md`

In [ ]:
outfile = "changelog_%s.md" % __version__
with open(outfile, "w") as of:
    of.write(message)

---------------------